## [3.4 异常](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#exceptions)

程序员必须总是留意程序中可能出现的错误。例子数不胜数：一个函数可能不会收到它预期的信息，必需的资源可能会丢失，或者网络上的连接可能丢失。在设计系统时，程序员必须预料到可能产生的异常情况并且采取适当地措施来处理它们。

处理程序中的错误没有单一的正确方式。为提供一些持久性服务而设计的程序，例如 Web 服务器 应该对错误健壮，将它们记录到日志中为之后考虑，而且在尽可能长的时间内继续接受新的请求。另一方面，Python 解释器通过立即终止以及打印错误信息来处理错误，便于程序员在错误发生时处理它。在任何情况下，程序员必须决定程序如何对异常条件做出反应。

异常是这一节的话题，它为程序的错误处理提供了通用的机制。产生异常是一种技巧，终止程序正常执行流，发射异常情况产生的信号，并直接返回到用于响应异常情况的程序的封闭部分。Python 解释器每次在检测到语句或表达式错误时抛出异常。用户也可以使用`raise`或`assert`语句来抛出异常。

**抛出异常。**异常是一个对象实例，它的类直接或间接继承自`BaseException`类。第一章引入的`assert`语句产生`AssertionError`类的异常。通常，异常实例可以使用`raise`语句来抛出。`raise`语句的通用形式在 [Python 文档](http://docs.python.org/py3k/reference/simple_stmts.html#raise)中描述。`raise`的最常见的作用是构造异常实例并抛出它。

In [ ]:
raise Exception('An error occurred')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
Exception: an error occurred

当异常产生时，当前代码块的语句不会继续执行。除非异常被解决了（下面会描述），解释器会直接返回到“读取-求值-打印”交互式循环中，或者在 Python 以文件参数启动的情况下会完全终止。此外，解释器会打印栈回溯，它是结构化的文本块，描述了执行分支中的一系列嵌套的活动函数，它们是异常产生的位置。在上面的例子中，文件名称`<stdin>`表示异常由用户在交互式会话中产生，而不是文件中的代码。

**处理异常。**异常可以使用封闭的`try`语句来处理。`try`语句由多个子句组成，第一个子句以`try`开始，剩下的以`except`开始。

In [ ]:
try:
    <try suite>
except <exception class> as <name>:
    <except suite>
...

当`try`语句执行时，`<try suite>`总是会立即执行。`except`子句组只在`<try suite>`执行过程中的异常产生时执行。每个`except`子句指定了需要处理的异常的特定类。例如，如果`<exception class>`是`AssertionError`，那么任何继承自`AssertionError`的类实例都会被处理，标识符`<name> `绑定到所产生的异常对象上，但是这个绑定在`<except suite>`之外并不有效。

例如，我们可以使用`try`语句来处理异常，在异常发生时将`x`绑定为`0`。

In [ ]:
try:
        x = 1/0
    except ZeroDivisionError as e:
        print('handling a', type(e))
        x = 0
handling a <class 'ZeroDivisionError'>

In [ ]:
x

`try`语句能够处理产生在函数体中的异常，函数在`<try suite>`中调用。当异常产生时，控制流会直接跳到最近的`try`语句的能够处理该异常类型的`<except suite>`的主体中。

In [ ]:
def invert(x):
        result = 1/x  # Raises a ZeroDivisionError if x is 0
        print('Never printed if x is 0')
        return result
def invert_safe(x):
        try:
            return invert(x)
        except ZeroDivisionError as e:
            return str(e)

In [ ]:
invert_safe(2)
Never printed if x is 0
0.5

In [ ]:
invert_safe(0)

这个例子表明，`invert`中的`print`表达式永远不会求值，反之，控制流跳到了`handler`中的`except`子句组中。将`ZeroDivisionError e`强制转为字符串会得到由`handler: 'division by zero'`返回的人类可读的字符串。

### 3.4.1 异常对象

异常对象本身就带有属性，例如在`assert`语句中的错误信息，以及有关异常产生处的信息。用户定义的异常类可以携带额外的属性。

在第一章中，我们实现了牛顿法来寻找任何函数的零点。下面的例子定义了一个异常类，无论何时`ValueError`出现，它都返回迭代改进过程中所发现的最佳猜测值。数学错误（`ValueError`的一种）在`sqrt`在负数上调用时产生。这个异常由抛出`IterImproveError`处理，它将牛顿迭代法的最新猜测值储存为参数。

首先，我们定义了新的类，继承自`Exception`。

In [ ]:
class IterImproveError(Exception):
        def __init__(self, last_guess):
            self.last_guess = last_guess

下面，我们定义了`IterImprove`，我们的通用迭代改进算法的一个版本。这个版本通过抛出`IterImproveError`异常，储存最新的猜测值来处理任何`ValueError`。像之前一样，`iter_improve`接受两个函数作为参数，每个函数都接受单一的数值参数。`update`函数返回新的猜测值，而`done`函数返回布尔值，表明改进是否收敛到了正确的值。

In [ ]:
def iter_improve(update, done, guess=1, max_updates=1000):
        k = 0
        try:
            while not done(guess) and k < max_updates:
                guess = update(guess)
                k = k + 1
            return guess
        except ValueError:
            raise IterImproveError(guess)

最后，我们定义了`find_root`，它返回`iter_improve`的结果。`iter_improve`应用于由`newton_update`返回的牛顿更新函数。`newton_update`定义在第一章，在这个例子中无需任何改变。`find_root`的这个版本通过返回它的最后一个猜测之来处理`IterImproveError`。

In [ ]:
def find_root(f, guess=1):
        def done(x):
            return f(x) == 0
        try:
            return iter_improve(newton_update(f), done, guess)
        except IterImproveError as e:
            return e.last_guess

考虑使用`find_root`来寻找`2 * x ** 2 + sqrt(x)`的零点。这个函数的一个零点是`0`，但是在任何负数上求解它会产生`ValueError`。我们第一章的牛顿法实现会产生异常，并且不能返回任何零点的猜测值。我们的修订版实现在错误之前返回了最新的猜测值。

In [ ]:
from math import sqrt
find_root(lambda x: 2*x*x + sqrt(x))

虽然这个近似值仍旧距离正确的答案`0`很远，一些应用更倾向于这个近似值而不是`ValueError`。

异常是另一个技巧，帮助我们将程序细节划分为模块化的部分。在这个例子中，Python 的异常机制允许我们分离迭代改进的逻辑，它在`try`子句组中没有发生改变，以及错误处理的逻辑，它出现在`except`子句中。我们也会发现，异常在使用 Python 实现解释器时是个非常实用的特性。

## [3.5 组合语言的解释器](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#interpreters-for-languages-with-combination)

运行在任何现代计算机上的软件都以多种编程语言写成。其中有物理语言，例如用于特定计算机的机器语言。这些语言涉及到基于独立储存位和原始机器指令的数据表示和控制。机器语言的程序员涉及到使用提供的硬件，为资源有限的计算构建系统和功能的高效实现。高阶语言构建在机器语言之上，隐藏了表示为位集的数据，以及表示为原始指令序列的程序的细节。这些语言拥有例如过程定义的组合和抽象的手段，它们适用于组织大规模的软件系统。

元语言抽象 -- 建立了新的语言 -- 并在所有工程设计分支中起到重要作用。它对于计算机编程尤其重要，因为我们不仅仅可以在编程中构想出新的语言，我们也能够通过构建解释器来实现它们。编程语言的解释器是一个函数，它在语言的表达式上调用，执行求解表达式所需的操作。

我们现在已经开始了技术之旅，通过这种技术，编程语言可以建立在其它语言之上。我们首先会为计算器定义解释器，它是一种受限的语言，和 Python 调用表达式具有相同的语法。我们之后会从零开始开发 Scheme 和 Logo 语言的解释器，它们都是 Lisp 的方言，Lisp 是现在仍旧广泛使用的第二老的语言。我们所创建的解释器，在某种意义上，会让我们使用 Logo 编写完全通用的程序。为了这样做，它会实现我们已经在这门课中开发的求值环境模型。

### 3.5.1 计算器

